In [1]:
import numpy as np
import os 
import cv2 
import time 
import argparse
import multiprocessing
import tensorflow as tf
#from matplotlib import pyplot as plt
import sys
#import six.moves.urllib as urllib
import tarfile
from PIL import Image
import cv2

In [2]:
sys.path.append("/Users/rohinisharma/anaconda/lib/python2.7/site-packages/tensorflow/models/research/object_detection/")
sys.path.append("..")

In [3]:
%matplotlib inline 
from utils import label_map_util
from utils import visualization_utils as vis_util

In [4]:
detection_graph =tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile("/Users/rohinisharma/software/ssd_mobilenet_v1_coco_11_06_2017/frozen_inference_graph1.pb", 
                        'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def,name='')

In [5]:
NUM_CLASSES=90
label_map = label_map_util.load_labelmap("/Users/rohinisharma/anaconda/lib/python2.7/site-packages/tensorflow/models/research/object_detection/data/mscoco_label_map.pbtxt")
categories = label_map_util.convert_label_map_to_categories(label_map, 
                                                            max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [6]:
cap = cv2.VideoCapture(1)
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        #input and output tensors for detection_graph
        image_tensor=detection_graph.get_tensor_by_name("image_tensor:0")
        detection_boxes = detection_graph.get_tensor_by_name("detection_boxes:0")
        detection_scores = detection_graph.get_tensor_by_name("detection_scores:0")
        detection_classes = detection_graph.get_tensor_by_name("detection_classes:0")
        num_detections = detection_graph.get_tensor_by_name("num_detections:0")
        while True :
            ret, image_np = cap.read()
            image_np_expanded = np.expand_dims(image_np,axis=0)
            (boxes,scores,classes,num)= sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
            vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8)
            cv2.imshow('object detection', cv2.resize(image_np, (800,600)))
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break
            